# Pydantic with ORM

The Config property orm_mode must be set to True.


In [290]:
from typing import List
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

Base = declarative_base()


class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))

    def __str__(self):
        msg = f'id={self.id} || public_key={self.public_key} || name={self.name} || domains={self.domains} '
        return msg



class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True



co_orm = CompanyOrm(
    id='123',
    public_key='foobar',
    name='Testing',
    domains=['example.com', 'foobar.com'],
)


print(co_orm)
print("---------")
co_model = CompanyModel.from_orm(co_orm)
print(co_model.dict())


id=123 || public_key=foobar || name=Testing || domains=['example.com', 'foobar.com'] 
---------
{'id': 123, 'public_key': 'foobar', 'name': 'Testing', 'domains': ['example.com', 'foobar.com']}


What if we want to use a reserved name, that ends with an underscore in SQLAlchemy

In [293]:
import typing

from pydantic import BaseModel, Field
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base


class MyModel(BaseModel):
    _metadata: typing.Dict[str, str]  = Field(alias='metadata_')

    class Config:
        orm_mode = True


Base = declarative_base()


class SQLModel(Base):
    __tablename__ = 'my_table'
    id = sa.Column('id', sa.Integer, primary_key=True)
    # 'metadata' is reserved by SQLAlchemy, hence the '_'
    metadata_ = sa.Column('metadata', sa.JSON)


sql_model = SQLModel(metadata_={'key': 'val'}, id=1)

pydantic_model = MyModel.from_orm(sql_model)

print(pydantic_model.dict())
#> {'metadata': {'key': 'val'}}
print(pydantic_model.dict(by_alias=True))
#> {'metadata_': {'key': 'val'}}

# notice the difference between the key name in both the cases

{}
{}


# Validations

In [297]:
from pydantic import BaseModel, ValidationError

data = {
    "id": "E15",
    "Fname": "Pushpak",
    "Lname": "Ruhil"
}


class readDict(BaseModel):
    id: int
    Fname: str
    Lname: str


try:
    parse = readDict(**data)
    print("after parsing: ", parse)
except ValidationError as e:
    print("error: ", str(e))  # .errors(), str(e)



error:  1 validation error for readDict
id
  value is not a valid integer (type=type_error.integer)


In [298]:
from pydantic import validator

# defining our own exception
class FirstLetterException(Exception):
    def __init__(self, msg):
        super().__init__(msg)


class customErrors(BaseModel):
    id: int
    Fname: str
    Lname: str

    @validator('Fname')
    def name_Validator(cls, val:str):
        if val[0].islower():
            raise FirstLetterException("First letter of the First name needs to be capital.")
        return val




In [300]:
data = {
    "id": "15",
    "Fname": "Pushpak",
    "Lname": "Ruhil"
}

print("after:", customErrors(**data))


after: id=15 Fname='Pushpak' Lname='Ruhil'


In [301]:
class customErrors(BaseModel):
    id: int
    Fname: str
    Lname: str

    @validator('Fname', 'Lname')
    def name_Validator(cls, all_vals:str, values, field):
        if all_vals[0].islower():
            raise FirstLetterException(f"First letter of the {field.name} needs to be capital. \"{all_vals}\" was provided")
        return all_vals



In [303]:

data = {
    "id": "15",
    "Fname": "pushpak",
    "Lname": "ruhil"
}

print("after:", customErrors(**data))


FirstLetterException: First letter of the Fname needs to be capital. "pushpak" was provided

https://towardsdatascience.com/battle-of-the-data-containers-which-python-typed-structure-is-the-best-6d28fde824e

A good guide for comparison between dataclass, pydantic, and other data containers

the guide states -> "Pydantic’s main purpose is parsing and using with ORMs, i.e. it is more suited to do sanitization/validation of data than being a mere container."

"However, before using pydantic you have to be sure that in fact, you require to sanitize data, as it will come with a performance hit, as you will see in the following sections."


In [304]:
print(customErrors.parse_file(r'data.json'))

id=15 Fname='Pushpak' Lname='Ruhil'


# Continue from here --->

# Dynamic models

when the shape of the data is not known until run-time

In [305]:
from pydantic import BaseModel, create_model
from typing import List

class FooModel(BaseModel):
    foo: str
    bar: int = 123


BarModel = create_model(
    'BarModel',
    apple='russet',
    banana='yellow',
    mango='12354',
    __base__=FooModel,
)
print(BarModel)
#> <class 'pydantic.main.BarModel'>

<class 'pydantic.main.BarModel'>


In [308]:
BarModel.__fields__

{'foo': ModelField(name='foo', type=str, required=True),
 'bar': ModelField(name='bar', type=int, required=False, default=123),
 'apple': ModelField(name='apple', type=str, required=False, default='russet'),
 'banana': ModelField(name='banana', type=str, required=False, default='yellow'),
 'mango': ModelField(name='mango', type=str, required=False, default='12354')}

In [307]:
for k, v in BarModel.__fields__.items():
    print(k, v.default)
    # print(type(v.default))

foo None
bar 123
apple russet
banana yellow
mango 12354


In [184]:
data = {
    'streak': '1',
    'tracker_count': '3',
    'member_since': '172 days',
    'trackers': {'28': ['Numbersss', 'num', '2022-07-06T19:16:44.994'],
                 '38': ['lets see', 'time_dur', '2022-03-06T23:17'],
                '52': ['lets see', 'bool', '2022-03-07T14:30']},
    'graph_path': 'static/images/homepage_pushpak.png'
}




class DashboardJSON(BaseModel):
    streak :  int
    tracker_count: int
    member_since: str
    graph_path: str
    class Config:
        extra = 'allow' # Allows extra attributes without defining them. Works great for dynamic data



In [182]:
trackers_data = create_model('trackers_Data', **data, __base__ = DashboardJSON)
for k,v in (trackers_data.__fields__.items()):
    print(k, v.default)


TypeError: The type of trackers_Data.streak differs from the new default value; if you wish to change the type of this field, please use a type annotation

In [199]:
print(DashboardJSON(**data).json(indent=3))

{
   "streak": 1,
   "tracker_count": 3,
   "member_since": "172 days",
   "graph_path": "static/images/homepage_pushpak.png",
   "trackers": {
      "28": [
         "Numbersss",
         "num",
         "2022-07-06T19:16:44.994"
      ],
      "38": [
         "lets see",
         "time_dur",
         "2022-03-06T23:17"
      ],
      "52": [
         "lets see",
         "bool",
         "2022-03-07T14:30"
      ]
   }
}


In [200]:
from pydantic import create_model, ValidationError, validator


class username_alphanumeric(Exception):
    def __init__(cls, msg):
        super().__init__(msg)

# ===============
def UsernameException(cls, v):
    if not v.isalnum():
        raise username_alphanumeric("Username must be alpha numeric")

our_validator = {
    'username_validator': validator('username')(UsernameException)
}

# ===============

UserModel = create_model(
    'UserModel',
    username=(str, ...),
    __validators__=our_validator
)

user = UserModel(username='scolvin')
print(user)


print("-------------------")

try:
    UserModel(username='scolvi##n')
except ValidationError as e:
    print(e)

username=None
-------------------


username_alphanumeric: Username must be alpha numeric

# Field Ordering

In [214]:

class Model(BaseModel):
    a: int
    b = 2
    c: int = 1
    d = 0
    e: int


print(Model.__fields__.keys())
print("-----------")
m = Model(e=3, a=1, x=10) # x is ignored
print(m.dict())
print("--------------------------------------------")
class Model2(BaseModel):
    a: int
    b:int = 2
    c: int = 1
    d:int = 0
    e: float

print(Model2.__fields__.keys())
print("-----------")
m = Model2(e=2, a=1)
print(m.dict())

dict_keys(['a', 'c', 'e', 'b', 'd'])
-----------
{'a': 1, 'c': 1, 'e': 3, 'b': 2, 'd': 0}
--------------------------------------------
dict_keys(['a', 'b', 'c', 'd', 'e'])
-----------
{'a': 1, 'b': 2, 'c': 1, 'd': 0, 'e': 2.0}


Field function to be more specific about fields ->

Field() -> https://pydantic-docs.helpmanual.io/usage/schema/#field-customization

# Root validator

In [220]:
from pydantic import BaseModel, ValidationError, root_validator


class UserModel(BaseModel):
    username: str
    password1: str
    password2: str

    @root_validator(pre=True)
    def check_card_number_omitted(cls, values):
        assert 'card_number' not in values, 'card_number should not be included'
        return values

    @root_validator
    def check_passwords_match(cls, values):
        pw1, pw2 = values.get('password1'), values.get('password2')
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError('passwords do not match')
        return values


print(UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn'))

print("--------------------------------------------")
print("--------------------------------------------")
try:
    UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn2')
except ValidationError as e:
    print(e)



print("--------------------------------------------")
print("--------------------------------------------")

try:
    UserModel(
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn',
        card_number='1234',
    )
except ValidationError as e:
    print(e)

username='scolvin' password1='zxcvbn' password2='zxcvbn'
--------------------------------------------
--------------------------------------------
1 validation error for UserModel
__root__
  passwords do not match (type=value_error)
--------------------------------------------
--------------------------------------------
1 validation error for UserModel
__root__
  card_number should not be included (type=assertion_error)


# Config sub-class

options available -> https://pydantic-docs.helpmanual.io/usage/model_config/#options

In [229]:
class Model(BaseModel):
    v: str

    class Config:
        title= "Hello there"
        # pass


In [230]:
print( Model(v="A string to be passed").schema_json(indent=2) )

{
  "title": "Hello there",
  "type": "object",
  "properties": {
    "v": {
      "title": "V",
      "type": "string"
    }
  },
  "required": [
    "v"
  ]
}


# Alias generator

In [233]:
def to_camel(string: str) -> str:
    return ''.join(word.capitalize() for word in string.split('_'))


class Voice(BaseModel):
    name: str
    language_code: str

    class Config:
        alias_generator = to_camel


voice = Voice(Name='Filiz', LanguageCode='tr-TR')
print(voice.__fields__.keys())

print(voice.dict(by_alias=True))

dict_keys(['name', 'language_code'])
{'Name': 'Filiz', 'LanguageCode': 'tr-TR'}


# Union of data types to allow multiple

In [234]:
from typing import Union

from pydantic import BaseModel


class Foo(BaseModel):
    pass


class Bar(BaseModel):
    pass


class Model(BaseModel):
    x: Union[str, int]
    y: Union[Foo, Bar]

    # class Config:
    #     smart_union = True


print(Model(x=1, y=Bar()))

x='1' y=Foo()


pydantic tries to validate in the order of the Union. So sometimes you may have unexpected coerced data.

This can be prevented using the class Config

# Pvt attributes

In [237]:
class Model(BaseModel):
    x: int
    y: int
    _pvt : str

test = Model(x=10, y=15, _pvt="this will be ingored, can only be set from the class")

In [239]:
test.json()

'{"x": 10, "y": 15}'

In [244]:
maps_data = {
  "destination_addresses": [
    "Washington, DC, USA",
    "Philadelphia, PA, USA",
    "Santa Barbara, CA, USA",
    "Miami, FL, USA",
    "Austin, TX, USA",
    "Napa County, CA, USA"
  ],
  "origin_addresses": [
    "New York, NY, USA"
  ],
  "rows": [{
    "elements": [{
        "distance": {
          "text": "227 mi",
          "value": 365468
        },
        "duration": {
          "text": "3 hours 54 mins",
          "value": 14064
        },
        "status": "OK"
      },
      {
        "distance": {
          "text": "94.6 mi",
          "value": 152193
        },
        "duration": {
          "text": "1 hour 44 mins",
          "value": 6227
        },
        "status": "OK"
      },
      {
        "distance": {
          "text": "2,878 mi",
          "value": 4632197
        },
        "duration": {
          "text": "1 day 18 hours",
          "value": 151772
        },
        "status": "OK"
      },
      {
        "distance": {
          "text": "1,286 mi",
          "value": 2069031
        },
        "duration": {
          "text": "18 hours 43 mins",
          "value": 67405
        },
        "status": "OK"
      },
      {
        "distance": {
          "text": "1,742 mi",
          "value": 2802972
        },
        "duration": {
          "text": "1 day 2 hours",
          "value": 93070
        },
        "status": "OK"
      },
      {
        "distance": {
          "text": "2,871 mi",
          "value": 4620514
        },
        "duration": {
          "text": "1 day 18 hours",
          "value": 152913
        },
        "status": "OK"
      }
    ]
  }],
  "status": "OK"
}

In [270]:

def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [271]:
map_result = json_extract(maps_data, 'text')




In [277]:
map_distance = map_result[0::2]
map_time = map_result[1::2]

In [284]:
for d,t in zip(map_distance, map_time):
    print(f'{t} required for a distance of {d}')

3 hours 54 mins required for a distance of 227 mi
1 hour 44 mins required for a distance of 94.6 mi
1 day 18 hours required for a distance of 2,878 mi
18 hours 43 mins required for a distance of 1,286 mi
1 day 2 hours required for a distance of 1,742 mi
1 day 18 hours required for a distance of 2,871 mi
